In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/kannada-mnist/Kannada_MNIST_datataset_paper/Kannada_MNIST_npz/Kannada_MNIST/y_kannada_MNIST_train.npz'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import math
import numpy as np
import pandas as pd
import h5py
import skimage
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
# import tensorflow as tf
# from tensorflow.python.framework import ops
# from cnn_utils import *

%matplotlib inline



In [ ]:
num_px = 28
my_image = "1.jpg"   
fname = "" + my_image
image = np.array(plt.imread(fname))
plt.imshow(image)
print(image.shape)
compressed_image = skimage.transform.resize(image, output_shape=(num_px,num_px))
# plt.imshow(compressed_image)
my_image = skimage.transform.resize(image, output_shape=(num_px,num_px)).reshape((1, num_px*num_px*3)).T

In [ ]:
train=pd.read_csv('../input/Kannada-MNIST/train.csv')
test=pd.read_csv('../input/Kannada-MNIST/test.csv')
sample_sub=pd.read_csv('../input/Kannada-MNIST/sample_submission.csv')
print("Train set shape = " +str(train.shape))
print("Test set shape = " +str(test.shape))
# print("Dif set shape = " +str(Dig_MNIST.shape))
train.head(10)
# test.head(3)

In [ ]:
test.head(3)
test=test.drop('id',axis=1)

In [ ]:
y=train.label.value_counts()
sns.barplot(y.index,y)

In [ ]:
X_train=train.drop('label',axis=1)
# X_test=test
# X_train.head(5)
Y_train=train.label
X_train=X_train/255
test=test/255
print(test.shape)
print(X_train.shape)
X_train=X_train.values.reshape(-1,28,28,1)
test=test.values.reshape(-1,28,28,1)
Y_train=to_categorical(Y_train)
print(X_train.shape)
print(Y_train.shape)

X_train,X_test,y_train,y_test=train_test_split(X_train,Y_train,random_state=42,test_size=0.15)



In [ ]:
# y_train.head
plt.imshow(X_train[6][:,:,0])

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 6

# input image dimensions
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)

# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
test=pd.read_csv('../input/Kannada-MNIST/test.csv')
test_id=test.id

test=test.drop('id',axis=1)
test=test/255
test=test.values.reshape(-1,28,28,1)
print(test.shape)

In [ ]:
y_pre=model.predict(test)     ##making prediction
y_pre=np.argmax(y_pre,axis=1)

In [ ]:
sample_sub['label']=y_pre
sample_sub.to_csv('submission.csv',index=False)
sample_sub.head()

In [ ]:
plt.imshow(test[3][:,:,0])